<a href="https://colab.research.google.com/github/guilleto/Introduction-to-Machine-Learning/blob/main/Discord_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a crear un bot de discord el cual lea el mensaje mediante una etiqueta y lo transforme utilizando la api gratis de Opanai (gpt 3)

*Update:* Se ha cambiado el provedor de la ia a interpretar el prompt por temas de pricing y tokens, pasando de openapi a un itermediario Eden Ai

iniciamos instalando las dependencias necesarias

In [1]:
!pip install discord.py
# !pip install openai
!pip install nest_asyncio
!pip install requests
!pip install  json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


Las importamos en el proyecto

In [2]:
import discord
# from openai import OpenAI
import nest_asyncio

#request for https Eden Ai
import requests
import json

Configuramos las credenciales de **Openai** y **Discord**

*Update:* **Openai** cambia a **Eden Ai**

In [7]:
# Configurar la clave del bot de Discord
DISCORD_TOKEN = 'dsaasd'

# Configurar la clave de la API de OpenAI
# clientOpenapi = OpenAI(api_key='asdads')

url = "https://api.edenai.run/v2/text/generation"

headers = {
    "Authorization": "Bearer asdasd",
    "accept": "application/json",
    "content-type": "application/json"
}


Definimos la funcion que va a procesar los mensajes mediante nuestro bot


In [4]:
# Codigo para realizar la coneccion directamente con openApi pero ya no tengo tokens activos

# def procesar_mensaje(mensaje):
#     # Verificar si el mensaje comienza con el tag $magic_message
#     if mensaje.startswith("$magic_message"):
#         # Eliminar el tag del mensaje
#         texto = mensaje.replace("$magic_message", "").strip()
#         # Generar una respuesta mágica utilizando GPT
#         respuesta = clientOpenapi.completions.create(
#             # model="gpt-3.5-turbo-0301",
#             model="curie",
#             prompt="Crea un mensaje el cual este orientado en un mundo de fantasia en epoca medieval donde halla magia y creaturasmiticas y cosas por el estilo basandose en este mensaje base: " + texto,
#         )
#         return respuesta.choices[0].text.strip()
#     else:
#         return None

previous_history = []

def procesar_mensaje(mensaje):
    # Verificar si el mensaje comienza con el tag $magic_message
    if mensaje.startswith("$magic_message"):
        texto = mensaje.replace("$magic_message", "").strip()
        prompt = "Crea un mensaje el cual este orientado en un mundo de fantasia en epoca medieval donde halla magia y criaturas miticas y cosas por el estilo basandose en este mensaje: " + texto

        payload = {
            "providers": "openai",
            "chatbot_global_action": "Actua como si estuvieras narrando una historia de fantasia",
            "previous_history": previous_history,
            "response_as_dict": True,
            "attributes_as_list": False,
            "show_original_response": False,
            "temperature": 0.5,
            "max_tokens": 200,
            "text": prompt,
            "fallback_providers": ""
        }
        # Se manda peticion https post con el payload de configuracion
        response = requests.post(url, json=payload, headers=headers)
        # Guardar la informacion de la historia en la sesion
        previous_history.append(response.text)
        result = json.loads(response.text)

        return result['openai']['generated_text']
    else:
        return None

Ponemos en funcionamiento el bot, para que escuche los eventos

In [5]:

nest_asyncio.apply()

# Especifica los intents que tu bot utilizará
intents = discord.Intents.default()
intents.typing = True
intents.message_content = True
intents.messages = True
# Crea el cliente de Discord con los intents especificados
client = discord.Client(intents=intents)




@client.event
async def on_message(message):
    print(message)
    if message.author == client.user:
        return
    print("contenido")
    print(message.content)
    respuesta = procesar_mensaje(message.content)
    if respuesta:
        await message.channel.send(respuesta)

### Ejecutamos el bot para que este en linea

In [6]:
client.run(DISCORD_TOKEN)

2024-02-06 20:36:44 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2024-02-06 20:36:45 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 0546ef77d244e2b2218ddef0e571f576).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: 0546ef77d244e2b2218ddef0e571f576).


<Message id=1204526282247905331 channel=<TextChannel id=349747192631918593 name='main' position=0 nsfw=True news=False category_id=None> type=<MessageType.default: 0> author=<Member id=329743127827054593 name='mrmemot' global_name='MrMemot' bot=False nick=None guild=<Guild id=349747192212226049 name='lolazos' shard_id=0 chunked=False member_count=89>> flags=<MessageFlags value=0>>
contenido
$magic_message <@213738160981671936> creo que ya quedó el generador de conversaciones, lo puse como una historia completa a ver qué tal
<Message id=1204526299419254784 channel=<TextChannel id=349747192631918593 name='main' position=0 nsfw=True news=False category_id=None> type=<MessageType.default: 0> author=<Member id=1204463465633419275 name='Magician_snoop' global_name=None bot=True nick=None guild=<Guild id=349747192212226049 name='lolazos' shard_id=0 chunked=False member_count=89>> flags=<MessageFlags value=0>>
<Message id=1204526463433580575 channel=<TextChannel id=349747192631918593 name='mai